In [3]:
!curl -Lso "train_img.gz" https://storage.googleapis.com/cvdf-datasets/mnist/train-images-idx3-ubyte.gz
!curl -Lso "train_label.gz" https://storage.googleapis.com/cvdf-datasets/mnist/train-labels-idx1-ubyte.gz
!curl -Lso "test_img.gz" https://storage.googleapis.com/cvdf-datasets/mnist/t10k-images-idx3-ubyte.gz
!curl -Lso "test_label.gz" https://storage.googleapis.com/cvdf-datasets/mnist/t10k-labels-idx1-ubyte.gz